In [1]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_3164/945712042.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [1]:
import pandas as pd
import os
import argparse
from typing import Any, Callable, Sequence
from pathlib import Path
from PIL import Image
import matplotlib.image as mpimg

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.flatten_util import ravel_pytree
import numpy as np
import matplotlib.pyplot as plt
from flax import linen as nn


from bong.util import run_rebayes_algorithm, gaussian_kl_div, MLP
from bong.src import bbb, blr, bog, bong, experiment_utils
from bong.agents import AGENT_NAMES
from plot_utils import *



Matplotlib created a temporary cache directory at /tmp/matplotlib-jqzopfjb because the default path (/teamspace/studios/this_studio/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [17]:
parser = argparse.ArgumentParser()

# Data parameters
parser.add_argument("--dataset", type=str, default="reg") 
parser.add_argument("--data_dim", type=int, default=10)
parser.add_argument("--data_key", type=int, default=0)
parser.add_argument("--dgp_type", type=str, default="lin") # or mlp
parser.add_argument("--dgp_neurons", type=int, nargs="+", default=[20, 20, 1]) 
parser.add_argument("--dgp_neurons_str", type=str, default="") # 20_20_1 
parser.add_argument("--emission_noise", type=float, default=1.0)
parser.add_argument("--ntrain", type=int, default=500)
parser.add_argument("--nval", type=int, default=500)
parser.add_argument("--ntest", type=int, default=1000)
parser.add_argument("--add_ones", type=bool, default=False)


# Model parameters
parser.add_argument("--agent", type=str, default="bong_fc", choices=AGENT_NAMES)
parser.add_argument("--agent_key", type=int, default=0)
parser.add_argument("--lr", type=float, default=0.01)
parser.add_argument("--niter", type=int, default=10) 
parser.add_argument("--nsample", type=int, default=10) 
parser.add_argument("--ef", type=int, default=0)
parser.add_argument("--linplugin", type=int, default=0)
parser.add_argument("--rank", type=int, default=10)
parser.add_argument("--model_type", type=str, default="lin") # or mlp
parser.add_argument("--model_neurons", type=int, nargs="+", default=[1])
parser.add_argument("--model_neurons_str", type=str, default="") 
parser.add_argument("--init_var", type=float, default=1.0)
parser.add_argument("--algo_key", type=int, default=0)

# results
parser.add_argument("--dir", type=str, default="", help="directory to store results") 
parser.add_argument("--debug", type=bool, default=False)


args = parser.parse_args([])

In [18]:
from datasets import *
from models import *
from bong.agents import *
from run_job import *

data = make_dataset(args)
print(data['X_tr'].shape)

model = make_model(args, data)

constructor = AGENT_DICT[args.agent]['constructor']
agent = constructor(
                    **model['model_kwargs'],
                    agent_key = args.agent_key,
                    learning_rate = args.lr,
                    num_iter = args.niter,
                    num_samples = args.nsample,
                    linplugin = args.linplugin,
                    empirical_fisher = args.ef,
                    rank = args.rank
                )
key = jr.PRNGKey(args.agent_key)
results, elapsed = run_agent(key, agent, data, model)
df = pd.DataFrame(results)
attributes = parse_agent_full_name(agent.name)
meta = { # non time-series data
    'data_name': data['name'],
    'model_name': model['name'],
    'agent_name': agent.name,
    'algo': attributes['algo'],
    'param': attributes['param'],
    'elapsed': elapsed
    }


(500, 10)
Running bong_fc-MC10-I1-LR0-EF1-Lin0 + mlp_10_1 on reg-D10-lin_1
Using GPU of type:  None
Time 15.75s
Test NLL: 1.5503,  NLPD: 1.5489
Val NLL 1.5632,  NLPD: 1.5948


In [20]:
from plot_utils import *

root_dir = '/teamspace/studios/this_studio/jobs'
data_dir = 'reg-D10-mlp_20_20_1'
model_dir = 'mlp_10_10_1'
agent_dir = 'A:Any-P:Any-Lin:1-LR:Any-IT:10-MC:10-EF:1-R:10'
dir = f'{root_dir}/{data_dir}/{model_dir}/{agent_dir}'



